# Obligatorio - Machine Learning



# Importación de librerías

In [ ]:
# Importaciones principales
import pandas as pd  # Manejo y análisis de datos estructurados (DataFrames)
import numpy as np  # Operaciones numéricas y manejo de arrays

# Visualización de datos
import matplotlib.pyplot as plt  # Visualización y gráficos

# Preprocesamiento y selección de características
from sklearn.preprocessing import LabelEncoder, StandardScaler  # Codificación de variables categóricas y escalado de datos
from sklearn.feature_selection import f_regression, SelectKBest  # Selección de características relevantes basadas en correlación

# Modelos de aprendizaje supervisado
from sklearn.linear_model import LinearRegression, Ridge  # Modelos lineales para regresión
from sklearn.tree import DecisionTreeRegressor  # Árboles de decisión para regresión
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor  # Modelos avanzados
from sklearn.neural_network import MLPRegressor  # Redes neuronales para regresión

# Evaluación y optimización
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score  # Dividir datos, búsqueda de hiperparámetros y validación cruzada
from sklearn.metrics import (  # Métricas de evaluación para modelos de regresión
    mean_squared_error,
    mean_absolute_error,
    median_absolute_error,
    r2_score,
    max_error,
)

# Text Mining
from sklearn.feature_extraction.text import TfidfVectorizer  # Representación numérica de texto (TF-IDF)

# Utilidades estadísticas y preprocesamiento avanzado
from scipy.stats.mstats import winsorize  # Manejo de valores extremos (outliers) mediante winsorización

# Herramientas de pipeline
from sklearn.pipeline import Pipeline  # Construcción de pipelines para flujos de preprocesamiento y modelado

# Weights and Biases
import wandb # Herramienta para el registro de experimentos y seguimiento de modelos

# Utilidades generales
import os
from math import sqrt  # Función matemática para la raíz cuadrada

# Cargar datasets
df_train = pd.read_csv('./dataset/train.csv')  # Cargar el conjunto de entrenamiento
df_test = pd.read_csv('./dataset/test.csv')  # Cargar el conjunto de prueba

# Preprocesamiento de datos

### Funciones Auxiliares

In [ ]:
# Paso 1: Rellenar valores faltantes
def clean_data(df):
    # Completar valores faltantes
    df['Year'] = df['Year'].fillna(df['Year'].median())  # Completar con la mediana
    df['Publisher'] = df['Publisher'].fillna('Unknown')  # Completar con "Unknown"

    # Eliminar columnas irrelevantes
    if 'Summary' in df.columns:
        df = df.drop(columns=['Summary'])

    return df

# Paso 2: Convertir columnas numéricas con valores inconsistentes
def convert_to_numeric(value):
    """
    Convierte valores como '1.2K' en valores numéricos.
    """
    value_str = str(value)
    if 'K' in value_str:
        return float(value_str.replace('K', '')) * 1000
    try:
        return float(value_str)
    except ValueError:
        return 0.0

# Paso 3: Manejar outliers
def handle_outliers(df, columns):
    """
    Aplica winsorización para manejar valores extremos en las columnas especificadas.
    """
    for col in columns:
        df[col] = winsorize(df[col], limits=[0.01, 0.01])  # Recorta 1% inferior y superior
    return df

# Paso 4: Codificar variables categóricas
def encode_categorical(df, categorical_columns):
    label_encoder = LabelEncoder()
    for col in categorical_columns:
        if col in df.columns:
            df[col] = label_encoder.fit_transform(df[col])
    return df

# Paso 5: Escalar columnas numéricas
def scale_numerical(df, numerical_columns):
    scaler = StandardScaler()
    df[numerical_columns] = scaler.fit_transform(df[numerical_columns])
    return df

# Paso 6: Rellenar valores faltantes en `Genre` usando `Summary`
def fill_missing_genres(df):
    """
    Rellena los valores faltantes de la columna `Genre` basándose en la columna `Summary`.
    Utiliza un modelo de clasificación de texto.
    """
    # Separar filas con y sin `Genre`
    df_with_genre = df[df['Genre'].notnull()]
    df_missing_genre = df[df['Genre'].isnull()]

    if len(df_missing_genre) > 0:
        # Convertir valores categóricos de `Genre` a números
        label_encoder = LabelEncoder()
        df_with_genre['Genre'] = label_encoder.fit_transform(df_with_genre['Genre'])

        # Modelo de clasificación de texto
        text_clf = Pipeline([
            ('tfidf', TfidfVectorizer(stop_words='english', max_features=1000)),
            ('clf', LogisticRegression(max_iter=1000, random_state=42))
        ])

        # Entrenar el modelo con los datos disponibles
        text_clf.fit(df_with_genre['Summary'], df_with_genre['Genre'])

        # Predecir los géneros faltantes
        predicted_genres = text_clf.predict(df_missing_genre['Summary'])

        # Convertir las predicciones de vuelta a etiquetas originales
        df_missing_genre['Genre'] = label_encoder.inverse_transform(predicted_genres)

        # Combinar las filas con `Genre` y las predichas
        df = pd.concat([df_with_genre, df_missing_genre])

# Paso 7: Crear nuevas columnas derivadas
def create_new_columns(df):
    """
    Crea nuevas columnas derivadas basadas en la información existente.
    """
    df['Game_Age'] = 2024 - df['Year']
    
    # Ratios regionales
    df['NorthAmerica_Global_Ratio'] = df['North America'] / df['Global']
    df['Europe_Global_Ratio'] = df['Europe'] / df['Global']
    df['Japan_Global_Ratio'] = df['Japan'] / df['Global']
    df['RestOfWorld_Global_Ratio'] = df['Rest of World'] / df['Global']

    # Relación entre reseñas y wishlist
    df['Reviews_Wishlist_Ratio'] = df['Number of Reviews'] / df['Wishlist']

    # Interacciones categóricas
    df['Publisher_Platform_Interaction'] = df['Publisher'] + df['Platform']

    return df


In [ ]:
def preprocess_data_with_genre_and_columns(df):
    # Limpiar datos
    df = clean_data(df)

    # Rellenar valores faltantes en `Genre` usando `Summary`
    if 'Summary' in df.columns and 'Genre' in df.columns:
        df = fill_missing_genres(df)

    # Convertir columnas numéricas con valores inconsistentes
    columns_to_convert = ['Europe', 'Japan', 'Rest of World', 'North America', 
                          'Global', 'Number of Reviews', 'Wishlist']
    for col in columns_to_convert:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_numeric)

    # Manejar outliers
    outlier_columns = ['Global', 'Wishlist']
    df = handle_outliers(df, outlier_columns)

    # Codificar columnas categóricas
    categorical_columns = ['Game Title', 'Publisher', 'Platform', 'Genre']
    df = encode_categorical(df, categorical_columns)

    # Crear nuevas columnas derivadas
    df = create_new_columns(df)

    # Escalar columnas numéricas
    numerical_columns = ['North America', 'Europe', 'Japan', 'Rest of World', 
                         'Global', 'Number of Reviews', 'Wishlist', 'Game_Age', 
                         'Europe_Global_Ratio', 'Japan_Global_Ratio', 
                         'NorthAmerica_Global_Ratio', 'RestOfWorld_Global_Ratio',
                         'Reviews_Wishlist_Ratio']
    df = scale_numerical(df, numerical_columns)

    # Eliminar columnas irrelevantes
    columns_to_drop = ['Game Title']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    # Eliminar las columnas regionales originales después de crear los ratios
    columns_to_drop = ['North America', 'Europe', 'Japan', 'Rest of World']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    return df

# Aplicar el preprocesamiento actualizado
df_train = preprocess_data_with_genre_and_columns(df_train)
df_test = preprocess_data_with_genre_and_columns(df_test)

# Verificar el resultado
print("Datos preprocesados con géneros rellenados y nuevas columnas (train):")
print(df_train.head())

# Analisis exploratorio

In [ ]:
# Función para evaluar correlación con la variable objetivo (para regresión)
def evaluate_feature_correlation(X, y):
    """
    Evalúa la correlación de cada característica con la variable objetivo.
    Devuelve un DataFrame con las puntuaciones.
    """
    selector = SelectKBest(score_func=f_regression, k='all')
    selector.fit(X, y)
    scores = selector.scores_

    # Crear un DataFrame con las puntuaciones
    feature_scores = pd.DataFrame({'Feature': X.columns, 'Score': scores})
    feature_scores = feature_scores.sort_values(by='Score', ascending=False)

    print("Importancia de características basada en correlación:")
    print(feature_scores)
    return feature_scores

# Función para evaluar la importancia de características usando un modelo
def evaluate_feature_importance_model(X, y):
    """
    Evalúa la importancia de características utilizando un RandomForestRegressor.
    Devuelve un DataFrame con las importancias.
    """
    model = RandomForestRegressor(random_state=42)
    model.fit(X, y)
    importances = model.feature_importances_

    # Crear un DataFrame con las importancias
    feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

    # Visualizar la importancia de características
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importances['Feature'], feature_importances['Importance'])
    plt.xlabel("Importancia")
    plt.ylabel("Características")
    plt.title("Importancia de características según RandomForest")
    plt.gca().invert_yaxis()
    plt.show()

    print("Importancia de características basada en RandomForest:")
    print(feature_importances)
    return feature_importances

# Selección de las características más relevantes
def select_top_features(X, y, k=10):
    """
    Selecciona las k características más relevantes utilizando SelectKBest.
    """
    selector = SelectKBest(score_func=f_regression, k=k)
    X_selected = selector.fit_transform(X, y)

    # Obtener las columnas seleccionadas
    selected_columns = X.columns[selector.get_support()]
    print(f"Características seleccionadas ({k} mejores): {selected_columns}")
    return X_selected, selected_columns

# Para df_train (como ya tienes)
X_train = df_train.drop(columns=['Rating'])  # Variables predictoras
y_train = df_train['Rating']  # Variable objetivo

# Evaluar y seleccionar características en df_train
correlation_scores = evaluate_feature_correlation(X_train, y_train)
importance_scores = evaluate_feature_importance_model(X_train, y_train)
X_train_selected, selected_columns = select_top_features(X_train, y_train, k=10)

# Actualizar el dataset de entrenamiento
df_train_selected = pd.DataFrame(X_train_selected, columns=selected_columns)
df_train_selected['Rating'] = y_train  # Añadir la columna objetivo para usar en el modelo

# Aplicar la misma selección de características al conjunto de prueba**
if 'Rating' in df_test.columns:
    X_test = df_test.drop(columns=['Rating'])  # Variables predictoras en el conjunto de prueba
    y_test = df_test['Rating']  # Variable objetivo en el conjunto de prueba
else:
    X_test = df_test
    y_test = None

# Seleccionar las mismas características en df_test usando `selected_columns`
X_test_selected = X_test[selected_columns]

# Actualizar el dataset de prueba
df_test_selected = pd.DataFrame(X_test_selected, columns=selected_columns)
if y_test is not None:
    df_test_selected['Rating'] = y_test  # Solo si `Rating` está disponible

### Proceso de nuevo los datos eliminando columnas irrelevantes

In [ ]:
def preprocess_data_optimized(df):

    # Eliminar columnas menos relevantes
    columns_to_drop = ['id', 'Publisher_Platform_Interaction', 'Publisher', 'Wishlist', 
                       'Number of Reviews', 'Reviews_Wishlist_Ratio']
    df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])

    return df

# Aplicar el preprocesamiento optimizado
df_train = preprocess_data_optimized(df_train)
df_test = preprocess_data_optimized(df_test)

# Verificar el resultado
print("Datos preprocesados optimizados (train):")
print(df_train.head())

In [ ]:
# Función para evaluar correlación con la variable objetivo (para regresión)
def evaluate_feature_correlation(X, y):
    """
    Evalúa la correlación de cada característica con la variable objetivo.
    Devuelve un DataFrame con las puntuaciones.
    """
    selector = SelectKBest(score_func=f_regression, k='all')
    selector.fit(X, y)
    scores = selector.scores_

    # Crear un DataFrame con las puntuaciones
    feature_scores = pd.DataFrame({'Feature': X.columns, 'Score': scores})
    feature_scores = feature_scores.sort_values(by='Score', ascending=False)

    print("Importancia de características basada en correlación:")
    print(feature_scores)
    return feature_scores

# Función para evaluar la importancia de características usando un modelo
def evaluate_feature_importance_model(X, y):
    """
    Evalúa la importancia de características utilizando un RandomForestRegressor.
    Devuelve un DataFrame con las importancias.
    """
    model = RandomForestRegressor(random_state=42)
    model.fit(X, y)
    importances = model.feature_importances_

    # Crear un DataFrame con las importancias
    feature_importances = pd.DataFrame({'Feature': X.columns, 'Importance': importances})
    feature_importances = feature_importances.sort_values(by='Importance', ascending=False)

    # Visualizar la importancia de características
    plt.figure(figsize=(10, 6))
    plt.barh(feature_importances['Feature'], feature_importances['Importance'])
    plt.xlabel("Importancia")
    plt.ylabel("Características")
    plt.title("Importancia de características según RandomForest")
    plt.gca().invert_yaxis()
    plt.show()

    print("Importancia de características basada en RandomForest:")
    print(feature_importances)
    return feature_importances

# Selección de las características más relevantes
def select_top_features(X, y, k=10):
    """
    Selecciona las k características más relevantes utilizando SelectKBest.
    """
    selector = SelectKBest(score_func=f_regression, k=k)
    X_selected = selector.fit_transform(X, y)

    # Obtener las columnas seleccionadas
    selected_columns = X.columns[selector.get_support()]
    print(f"Características seleccionadas ({k} mejores): {selected_columns}")
    return X_selected, selected_columns

# Para df_train (como ya tienes)
X_train = df_train.drop(columns=['Rating'])  # Variables predictoras
y_train = df_train['Rating']  # Variable objetivo

# Evaluar y seleccionar características en df_train
correlation_scores = evaluate_feature_correlation(X_train, y_train)
importance_scores = evaluate_feature_importance_model(X_train, y_train)
X_train_selected, selected_columns = select_top_features(X_train, y_train, k=10)

# Actualizar el dataset de entrenamiento
df_train_selected = pd.DataFrame(X_train_selected, columns=selected_columns)
df_train_selected['Rating'] = y_train  # Añadir la columna objetivo para usar en el modelo

# Aplicar la misma selección de características al conjunto de prueba
# Como no tiene la columna `Rating`, simplemente seleccionamos las columnas relevantes
X_test = df_test[selected_columns]

# Actualizar el dataset de prueba
df_test_selected = pd.DataFrame(X_test, columns=selected_columns)

# df_test_selected ahora contiene solo las características seleccionadas para hacer predicciones
print("Características seleccionadas en el conjunto de prueba:")
print(df_test_selected.head())

# Division de datos procesados

In [ ]:
# Dividir los datos para entrenamiento y prueba

# Entrenamiento: X_train y y_train
X_train = df_train_selected.drop(columns=['Rating'])  # Variables predictoras del conjunto de entrenamiento
y_train = df_train_selected['Rating']  # Variable objetivo del conjunto de entrenamiento

# Prueba: X_test (sin Rating)
X_test = df_test_selected  # df_test seleccionado y procesado ya no incluye la columna Rating

# Entrenamiento

In [ ]:
def train_model(estimator, param_grid, X_train, y_train):
    """
    Entrena un modelo con los hiperparámetros especificados utilizando GridSearchCV.
    """
    grid_search = GridSearchCV(
        estimator,
        param_grid,
        cv=5,
        scoring='neg_mean_squared_error',
        verbose=1,
        n_jobs=-1
    )
    grid_search.fit(X_train, y_train)
    best_model = grid_search.best_estimator_
    print(f"Mejores parámetros: {grid_search.best_params_}")
    return best_model

# Evaluacion

In [ ]:
def evaluate_model(model, X_train, y_train, X_test):
    """
    Evalúa un modelo entrenado y genera las métricas y predicciones.
    """
    # Validación cruzada en el conjunto de entrenamiento
    cv_scores_mse = cross_val_score(model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
    cv_mean_mse = -cv_scores_mse.mean()
    cv_std_mse = cv_scores_mse.std()
    cv_rmse = sqrt(cv_mean_mse)
    mae = mean_absolute_error(y_train, model.predict(X_train))  # MAE en el conjunto de entrenamiento
    median_ae = median_absolute_error(y_train, model.predict(X_train))  # Mediana del error absoluto
    r2 = r2_score(y_train, model.predict(X_train))  # R² en el conjunto de entrenamiento
    max_error_value = max_error(y_train, model.predict(X_train))  # Máximo error absoluto

    print("\nValidación cruzada:")
    print(f"MSE promedio (CV): {cv_mean_mse:.4f}")
    print(f"RMSE promedio (CV): {cv_rmse:.4f}")
    print(f"Desviación estándar (MSE): {cv_std_mse:.4f}")

    # Generar predicciones para el conjunto de prueba
    y_pred = model.predict(X_test)

    metrics = {
        'cv_mean_mse': cv_mean_mse,
        'cv_std_mse': cv_std_mse,
        'cv_rmse': cv_rmse,
        'mae': mae,
        'median_ae': median_ae,
        'r2': r2,
        'max_error': max_error_value
    }
    return y_pred, metrics

# Resultados 

In [ ]:
#Generar resultados
def generate_results(y_pred, metrics, model_name, hyperparameters):
    """
    Genera las carpetas, archivos de predicciones y métricas.
    """
    # Crear carpetas base para resultados
    os.makedirs(f"predictions/{model_name}", exist_ok=True)
    os.makedirs(f"metrics/{model_name}", exist_ok=True)
    os.makedirs("metrics/MejoresModelos", exist_ok=True)

    # Guardar predicciones
    file_suffix = "_".join([f"{key}_{value}" for key, value in hyperparameters.items()])
    predictions_file = f"predictions/{model_name}/predicciones_{file_suffix}.csv"
    pd.DataFrame({
        "id": range(1, len(y_pred) + 1),  # Comenzar desde 1
        "predictions": y_pred
    }).to_csv(predictions_file, index=False)
    print(f"Predicciones guardadas en {predictions_file}")

    # Guardar métricas
    metrics_file = f"metrics/{model_name}/metrics_{file_suffix}.csv"
    with open(metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{metrics['cv_mean_mse']:.4f}\n")
        f.write(f"RMSE promedio (CV),{metrics['cv_rmse']:.4f}\n")
        f.write(f"Desviación estándar (MSE),{metrics['cv_std_mse']:.4f}\n")
        f.write(f"MAE promedio (CV),{metrics['mae']:.4f}\n")
        f.write(f"Mediana del Error Absoluto (MedianAE),{metrics['median_ae']:.4f}\n")
        f.write(f"R² (CV),{metrics['r2']:.4f}\n")
        f.write(f"Max Error (CV),{metrics['max_error']:.4f}\n")
    print(f"Métricas guardadas en {metrics_file}")

    return predictions_file, metrics_file

# Guarda el modelo con mejores metricas
def save_best_model_metrics(best_model, model_name, best_metrics, hyperparameters):
    """
    Guarda las métricas del mejor modelo en la carpeta MejoresModelos.
    """
    os.makedirs("metrics/MejoresModelos", exist_ok=True)

    # Crear un archivo con las métricas del mejor modelo
    file_suffix = "_".join([f"{key}_{value}" for key, value in hyperparameters.items()])
    best_metrics_file = f"metrics/MejoresModelos/{model_name}_mejor_modelo_metrics.csv"

    with open(best_metrics_file, "w") as f:
        f.write("Métrica,Valor\n")
        f.write(f"MSE promedio (CV),{best_metrics['cv_mean_mse']:.4f}\n")
        f.write(f"RMSE promedio (CV),{best_metrics['cv_rmse']:.4f}\n")
        f.write(f"Desviación estándar (MSE),{best_metrics['cv_std_mse']:.4f}\n")
        f.write(f"MAE promedio (CV),{best_metrics['mae']:.4f}\n")
        f.write(f"Mediana del Error Absoluto (MedianAE),{best_metrics['median_ae']:.4f}\n")
        f.write(f"R² (CV),{best_metrics['r2']:.4f}\n")
        f.write(f"Max Error (CV),{best_metrics['max_error']:.4f}\n")
    print(f"Métricas del mejor modelo guardadas en {best_metrics_file}")

# Flujo Principal y Ejecucion

In [ ]:
wandb.init(
    project="OBL-Machine-Learning-2024",
    name="Evaluación_Modelos",
    config={"task": "Regresión"}
)

Random Forest

In [ ]:
def main_rf(X_train, y_train, X_test):
    # Inicializar un nuevo experimento en WandB
    wandb.init(project="OBL-Machine-Learning-2024", name="RandomForest", config={"task": "Regression"})

    results = []
    n_estimators_values = [50, 100, 200]
    max_depth_values = [10, 20, None]
    max_features = ['sqrt']  # sqrt es recomendado para datasets grandes
    min_samples_split = [2, 5, 10]  # Controla el mínimo de datos requeridos para dividir un nodo

    for n_estimators in n_estimators_values:
        for max_depth in max_depth_values:
            # Entrenamiento
            param_grid = {
                'n_estimators': [n_estimators],
                'max_depth': [max_depth],
                'max_features': max_features,
                'min_samples_split': min_samples_split
            }
            model = train_model(RandomForestRegressor(random_state=42), param_grid, X_train, y_train)

            # Evaluación
            y_pred, metrics = evaluate_model(model, X_train, y_train, X_test)

            # Generar resultados
            hyperparameters = {"n_estimators": n_estimators, "max_depth": max_depth}
            predictions_file, metrics_file = generate_results(y_pred, metrics, "RandomForest", hyperparameters)

            # Guardar resultados intermedios
            results.append((n_estimators, max_depth, metrics['cv_mean_mse'], metrics['cv_std_mse'], 
                            predictions_file, metrics_file, model, metrics))

    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])  # x[2] es `cv_mean_mse`

    print("\nMejor modelo:")
    print(f"- Número de árboles (n_estimators): {best_model[0]}")
    print(f"- Profundidad máxima (max_depth): {best_model[1]}")
    print(f"- MSE promedio (CV): {best_model[2]:.4f}")
    print(f"- Desviación estándar del MSE (CV): {best_model[3]:.4f}")
    print(f"- MAE (Conjunto de prueba): {best_model[7]['mae']:.4f}")
    print(f"- R² (Conjunto de prueba): {best_model[7]['r2']:.4f}")
    print(f"- RMSE (Conjunto de prueba): {best_model[7]['cv_rmse']:.4f}")

    # Guardar métricas del mejor modelo
    hyperparameters = {"n_estimators": best_model[0], "max_depth": best_model[1]}
    save_best_model_metrics(best_model, "RandomForest", best_model[7], hyperparameters)

    # Registrar el mejor modelo en WandB
    wandb.log({
        "Mejor MSE promedio (CV)": best_model[2],
        "Mejor Desviación estándar (CV)": best_model[3],
        "Mejor RMSE (CV)": best_model[7]['cv_rmse'],
        "Mejor MAE": best_model[7]['mae'],
        "Mejor R²": best_model[7]['r2']
    })

    '''
    # Crear una tabla con los resultados numéricos del mejor modelo
    table_data = [
        [best_model[2],  # MSE promedio (CV)
         best_model[3],  # Desviación estándar (CV)
         best_model[7]['mae'],  # MAE
         best_model[7]['r2'],  # R²
         best_model[7]['cv_rmse']]  # RMSE
    ]
    table_columns = ["MSE promedio (CV)", "Desviación estándar (CV)", "MAE", "R²", "RMSE"]
    wandb.log({"Mejor Modelo": wandb.Table(data=table_data, columns=table_columns)})
    '''
    # Finalizar el experimento en WandB
    wandb.finish()

    return best_model

# Llamar a la función principal
best_model_rf = main_rf(X_train, y_train, X_test)

Gradient Boosting

In [ ]:
import wandb  # Asegúrate de importar WandB

def main_gb(X_train, y_train, X_test):
    # Inicializar un nuevo experimento en WandB
    wandb.init(project="OBL-Machine-Learning-2024", name="GradientBoosting", config={"task": "Regression"})

    results = []
    n_estimators_values = [100, 200]
    learning_rate_values = [0.1, 0.2]
    max_depth_values = [3, 5, 10]
    subsample_values = [0.8, 1.0]  # Fracción de muestras utilizadas para ajustar los árboles

    for n_estimators in n_estimators_values:
        for learning_rate in learning_rate_values:
            # Entrenamiento
            param_grid = {
                'n_estimators': [n_estimators],
                'learning_rate': [learning_rate],
                'max_depth': max_depth_values,
                'subsample': subsample_values
            }
            model = train_model(GradientBoostingRegressor(random_state=42), param_grid, X_train, y_train)

            # Evaluación
            y_pred, metrics = evaluate_model(model, X_train, y_train, X_test)

            # Generar resultados
            hyperparameters = {"n_estimators": n_estimators, "learning_rate": learning_rate}
            predictions_file, metrics_file = generate_results(y_pred, metrics, "GradientBoosting", hyperparameters)

            # Guardar resultados intermedios
            results.append((n_estimators, learning_rate, metrics['cv_mean_mse'], metrics['cv_std_mse'], 
                            predictions_file, metrics_file, model, metrics))

    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])  # x[2] es `cv_mean_mse`
    print("\nMejor modelo:")
    print(f"- Número de estimadores (n_estimators): {best_model[0]}")
    print(f"- Tasa de aprendizaje (learning_rate): {best_model[1]}")
    print(f"- MSE promedio (CV): {best_model[2]:.4f}")
    print(f"- Desviación estándar del MSE (CV): {best_model[3]:.4f}")
    print(f"- MAE (Conjunto de prueba): {best_model[7]['mae']:.4f}")
    print(f"- R² (Conjunto de prueba): {best_model[7]['r2']:.4f}")
    print(f"- RMSE (Conjunto de prueba): {best_model[7]['cv_rmse']:.4f}")

    # Guardar métricas del mejor modelo
    hyperparameters = {"n_estimators": best_model[0], "learning_rate": best_model[1]}
    save_best_model_metrics(best_model, "GradientBoosting", best_model[7], hyperparameters)

    # Registrar el mejor modelo en WandB
    wandb.log({
        "Mejor MSE promedio (CV)": best_model[2],
        "Mejor Desviación estándar (CV)": best_model[3],
        "Mejor RMSE (CV)": best_model[7]['cv_rmse'],
        "Mejor MAE": best_model[7]['mae'],
        "Mejor R²": best_model[7]['r2']
    })

    '''
    # Crear una tabla con los resultados numéricos del mejor modelo
    table_data = [
        [best_model[2],  # MSE promedio (CV)
         best_model[3],  # Desviación estándar (CV)
         best_model[7]['mae'],  # MAE
         best_model[7]['r2'],  # R²
         best_model[7]['cv_rmse']]  # RMSE
    ]
    table_columns = ["MSE promedio (CV)", "Desviación estándar (CV)", "MAE", "R²", "RMSE"]
    wandb.log({"Mejor Modelo": wandb.Table(data=table_data, columns=table_columns)})
    '''
    
    # Finalizar el experimento en WandB
    wandb.finish()

    return best_model

# Llamar a la función principal
best_model_gb = main_gb(X_train, y_train, X_test)

Ada boosting

In [ ]:
def main_ada(X_train, y_train, X_test):
    """
    Entrena, evalúa y guarda resultados para el modelo AdaBoost.
    """

    # Inicializar un nuevo experimento en WandB
    wandb.init(project="OBL-Machine-Learning-2024", name="AdaBoost", config={"task": "Regression"})
    
    results = []
    n_estimators_values = [50, 100]
    learning_rate_values = [0.1, 0.2, 0.5]

    for n_estimators in n_estimators_values:
        for learning_rate in learning_rate_values:
            # Configuración de hiperparámetros específicos de AdaBoost
            param_grid = {
                'n_estimators': [n_estimators],
                'learning_rate': [learning_rate],
                'estimator__max_depth': [5, 7, 10],  # Este hiperparámetro no cambiará
                'loss': ['linear', 'square', 'exponential']
            }

            # Crear el modelo base de AdaBoost con DecisionTreeRegressor
            base_estimator = DecisionTreeRegressor(random_state=42)
            model = AdaBoostRegressor(estimator=base_estimator, random_state=42)

            # Entrenamiento
            trained_model = train_model(model, param_grid, X_train, y_train)

            # Evaluación
            y_pred, metrics = evaluate_model(trained_model, X_train, y_train, X_test)

            # Generar resultados
            hyperparameters = {"n_estimators": n_estimators, "learning_rate": learning_rate}
            predictions_file, metrics_file = generate_results(y_pred, metrics, "AdaBoost", hyperparameters)

            # Guardar resultados intermedios
            results.append((n_estimators, learning_rate, metrics['cv_mean_mse'], metrics['cv_std_mse'], 
                            predictions_file, metrics_file, trained_model, metrics))

    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])  # x[2] es `cv_mean_mse`
    print("\nMejor modelo:")
    print(f"- Número de estimadores (n_estimators): {best_model[0]}")
    print(f"- Tasa de aprendizaje (learning_rate): {best_model[1]}")
    print(f"- MSE promedio (CV): {best_model[2]:.4f}")
    print(f"- Desviación estándar del MSE (CV): {best_model[3]:.4f}")
    print(f"- MAE (Conjunto de prueba): {best_model[7]['mae']:.4f}")
    print(f"- R² (Conjunto de prueba): {best_model[7]['r2']:.4f}")
    print(f"- RMSE (Conjunto de prueba): {best_model[7]['cv_rmse']:.4f}")

    # Guardar métricas del mejor modelo
    hyperparameters = {"n_estimators": best_model[0], "learning_rate": best_model[1]}
    save_best_model_metrics(best_model, "AdaBoost", best_model[7], hyperparameters)

    # Registrar el mejor modelo en WandB
    wandb.log({
        "Mejor MSE promedio (CV)": best_model[2],
        "Mejor Desviación estándar (CV)": best_model[3],
        "Mejor RMSE (CV)": best_model[7]['cv_rmse'],
        "Mejor MAE": best_model[7]['mae'],
        "Mejor R²": best_model[7]['r2']
    })

    '''
    #Crear una tabla con los resultados numéricos del mejor modelo
    table_data = [
        [best_model[2],  # MSE promedio (CV)
         best_model[3],  # Desviación estándar (CV)
         best_model[7]['mae'],  # MAE
         best_model[7]['r2'],  # R²
         best_model[7]['cv_rmse']]  # RMSE
    ]
    table_columns = ["MSE promedio (CV)", "Desviación estándar (CV)", "MAE", "R²", "RMSE"]
    wandb.log({"Mejor Modelo": wandb.Table(data=table_data, columns=table_columns)})
    '''

    # Finalizar el experimento en WandB
    wandb.finish()

    return best_model

best_model_ada = main_ada(X_train, y_train, X_test)

Decision Trees

In [ ]:
def main_tree(X_train, y_train, X_test):

    # Inicializar un nuevo experimento en WandB
    wandb.init(project="OBL-Machine-Learning-2024", name="ArbolesDeDecision", config={"task": "Regression"})

    results = []
    max_depth_values = [3, 5, 10, None]
    criterion_values = ['squared_error', 'absolute_error']
    max_features_values = ['sqrt', 'log2']

    for max_depth in max_depth_values:
        for criterion in criterion_values:
            for max_features in max_features_values:
                param_grid = {
                    'max_depth': [max_depth],
                    'criterion': [criterion],
                    'max_features': [max_features],
                    'min_samples_leaf': [1, 5, 10],
                    'splitter': ['best']
                }

                # Entrenamiento
                model = train_model(DecisionTreeRegressor(random_state=42), param_grid, X_train, y_train)

                # Evaluación
                y_pred, metrics = evaluate_model(model, X_train, y_train, X_test)

                # Generar resultados
                hyperparameters = {"max_depth": max_depth, "criterion": criterion, "max_features": max_features}
                predictions_file, metrics_file = generate_results(y_pred, metrics, "ArbolesDeDecision", hyperparameters)

                # Guardar resultados intermedios
                results.append((max_depth, criterion, max_features, metrics['cv_mean_mse'], metrics['cv_std_mse'], 
                                predictions_file, metrics_file, metrics))

    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[3])  # x[3] es `cv_mean_mse`
    print("\nMejor modelo:")
    print(f"- Profundidad máxima (max_depth): {best_model[0]}")
    print(f"- Criterio de división (criterion): {best_model[1]}")
    print(f"- Máximas características (max_features): {best_model[2]}")
    print(f"- MSE promedio (CV): {best_model[3]:.4f}")
    print(f"- Desviación estándar del MSE (CV): {best_model[4]:.4f}")
    print(f"- MAE (Conjunto de prueba): {best_model[7]['mae']:.4f}")
    print(f"- R² (Conjunto de prueba): {best_model[7]['r2']:.4f}")
    print(f"- RMSE (Conjunto de prueba): {best_model[7]['cv_rmse']:.4f}")

    # Registrar el mejor modelo en WandB
    wandb.log({
        "Mejor MSE promedio (CV)": best_model[2],
        "Mejor Desviación estándar (CV)": best_model[3],
        "Mejor RMSE (CV)": best_model[7]['cv_rmse'],
        "Mejor MAE": best_model[7]['mae'],
        "Mejor R²": best_model[7]['r2']
    })

    '''
    #Crear una tabla con los resultados numéricos del mejor modelo
    table_data = [
        [best_model[2],  # MSE promedio (CV)
         best_model[3],  # Desviación estándar (CV)
         best_model[7]['mae'],  # MAE
         best_model[7]['r2'],  # R²
         best_model[7]['cv_rmse']]  # RMSE
    ]
    table_columns = ["MSE promedio (CV)", "Desviación estándar (CV)", "MAE", "R²", "RMSE"]
    wandb.log({"Mejor Modelo": wandb.Table(data=table_data, columns=table_columns)})
    '''

    # Finalizar el experimento en WandB
    wandb.finish()

    # Guardar métricas del mejor modelo
    hyperparameters = {"max_depth": best_model[0], "criterion": best_model[1], "max_features": best_model[2]}
    save_best_model_metrics(best_model, "ArbolesDeDecision", best_model[7], hyperparameters)

    return best_model

best_model_tree = main_tree(X_train, y_train, X_test)

Redes Neuronales

In [ ]:
def main_nn(X_train, y_train, X_test):
    """
    Proceso principal para entrenar y evaluar Redes Neuronales.
    """

    # Inicializar un nuevo experimento en WandB
    wandb.init(project="OBL-Machine-Learning-2024", name="RedesNeuronales", config={"task": "Regression"})

    results = []
    hidden_layer_sizes_values = [(50,), (100,), (100, 50)]
    learning_rate_values = [0.001, 0.01]

    for hidden_layer_sizes in hidden_layer_sizes_values:
        for learning_rate_init in learning_rate_values:
            # Configuración del grid para Redes Neuronales
            param_grid = {
                'hidden_layer_sizes': [hidden_layer_sizes],
                'activation': ['relu'],  # Fijo
                'alpha': [0.0001, 0.001],  # Regularización fija
                'learning_rate_init': [learning_rate_init],
                'max_iter': [200, 300]
            }

            # Entrenar el modelo
            model = train_model(MLPRegressor(random_state=42), param_grid, X_train, y_train)

            # Evaluar el modelo
            y_pred, metrics = evaluate_model(model, X_train, y_train, X_test)

            # Generar resultados
            hyperparameters = {"hidden_layer_sizes": hidden_layer_sizes, "learning_rate_init": learning_rate_init}
            predictions_file, metrics_file = generate_results(y_pred, metrics, "RedesNeuronales", hyperparameters)

            # Guardar resultados intermedios
            results.append((hidden_layer_sizes, learning_rate_init, metrics['cv_mean_mse'], metrics['cv_std_mse'], 
                            predictions_file, metrics_file, metrics))

    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])  # x[2] es `cv_mean_mse`
    print("\nMejor modelo:")
    print(f"- Arquitectura oculta (hidden_layer_sizes): {best_model[0]}")
    print(f"- Tasa de aprendizaje (learning_rate_init): {best_model[1]}")
    print(f"- MSE promedio (CV): {best_model[2]:.4f}")
    print(f"- Desviación estándar del MSE (CV): {best_model[3]:.4f}")
    print(f"- MAE (Conjunto de prueba): {best_model[6]['mae']:.4f}")
    print(f"- R² (Conjunto de prueba): {best_model[6]['r2']:.4f}")
    print(f"- RMSE (Conjunto de prueba): {best_model[6]['cv_rmse']:.4f}")

    # Guardar las métricas del mejor modelo
    hyperparameters = {"hidden_layer_sizes": best_model[0], "learning_rate_init": best_model[1]}
    save_best_model_metrics(best_model, "RedesNeuronales", best_model[6], hyperparameters)

    # Registrar el mejor modelo en WandB
    wandb.log({
        "Mejor MSE promedio (CV)": best_model[2],
        "Mejor Desviación estándar (CV)": best_model[3],
        "Mejor RMSE (CV)": best_model[6]['cv_rmse'],
        "Mejor MAE": best_model[6]['mae'],
        "Mejor R²": best_model[6]['r2']
    })

    '''
    # Crear una tabla con los resultados numéricos del mejor modelo
    table_data = [
        [best_model[2],  # MSE promedio (CV)
         best_model[3],  # Desviación estándar (CV)
         best_model[6]['mae'],  # MAE
         best_model[6]['r2'],  # R²
         best_model[6]['cv_rmse']]  # RMSE
    ]
    table_columns = ["MSE promedio (CV)", "Desviación estándar (CV)", "MAE", "R²", "RMSE"]
    wandb.log({"Mejor Modelo": wandb.Table(data=table_data, columns=table_columns)})
    '''

    # Finalizar el experimento en WandB
    wandb.finish()

    return best_model

# Llamar al proceso principal
best_model_nn = main_nn(X_train, y_train, X_test)

Regresion Lineal con Ridge

In [ ]:
def main_ridge(X_train, y_train, X_test):
    """
    Proceso principal para entrenar y evaluar Ridge Regression.
    """

    # Inicializar un nuevo experimento en WandB
    wandb.init(project="OBL-Machine-Learning-2024", name="RidgeRegression", config={"task": "Regression"})

    results = []
    alpha_values = [0.1, 1, 10, 100]
    fit_intercept_values = [True, False]

    for alpha in alpha_values:
        for fit_intercept in fit_intercept_values:
            # Configuración de hiperparámetros
            param_grid = {'alpha': [alpha], 'fit_intercept': [fit_intercept]}

            # Entrenar el modelo
            model = train_model(Ridge(), param_grid, X_train, y_train)

            # Evaluar el modelo
            y_pred, metrics = evaluate_model(model, X_train, y_train, X_test)

            # Guardar resultados
            hyperparameters = {"alpha": alpha, "fit_intercept": fit_intercept}
            predictions_file, metrics_file = generate_results(y_pred, metrics, "RidgeRegression", hyperparameters)

            # Guardar resultados intermedios
            results.append((alpha, fit_intercept, metrics['cv_mean_mse'], metrics['cv_std_mse'], 
                            predictions_file, metrics_file, metrics))

    # Seleccionar el mejor modelo basado en MSE promedio
    best_model = min(results, key=lambda x: x[2])  # x[2] es `cv_mean_mse`
    print("\nMejor modelo:")
    print(f"- Alpha: {best_model[0]}")
    print(f"- Fit Intercept: {best_model[1]}")
    print(f"- MSE promedio (CV): {best_model[2]:.4f}")
    print(f"- Desviación estándar del MSE (CV): {best_model[3]:.4f}")
    print(f"- MAE (Conjunto de prueba): {best_model[6]['mae']:.4f}")
    print(f"- R² (Conjunto de prueba): {best_model[6]['r2']:.4f}")
    print(f"- RMSE (Conjunto de prueba): {best_model[6]['cv_rmse']:.4f}")

    # Guardar las métricas del mejor modelo
    hyperparameters = {"alpha": best_model[0], "fit_intercept": best_model[1]}
    save_best_model_metrics(best_model, "RidgeRegression", best_model[6], hyperparameters)

    # Registrar el mejor modelo en WandB
    wandb.log({
        "Mejor MSE promedio (CV)": best_model[2],
        "Mejor Desviación estándar (CV)": best_model[3],
        "Mejor RMSE (CV)": best_model[6]['cv_rmse'],
        "Mejor MAE": best_model[6]['mae'],
        "Mejor R²": best_model[6]['r2']
    })

    '''
    # Crear una tabla con los resultados numéricos del mejor modelo
    table_data = [
        [best_model[2],  # MSE promedio (CV)
         best_model[3],  # Desviación estándar (CV)
         best_model[6]['mae'],  # MAE
         best_model[6]['r2'],  # R²
         best_model[6]['cv_rmse']]  # RMSE
    ]
    table_columns = ["MSE promedio (CV)", "Desviación estándar (CV)", "MAE", "R²", "RMSE"]
    wandb.log({"Mejor Modelo": wandb.Table(data=table_data, columns=table_columns)})
    '''

    # Finalizar el experimento en WandB
    wandb.finish()

    return best_model

# Llamar a la función principal
best_model_ridge = main_ridge(X_train, y_train, X_test)